<a href="https://colab.research.google.com/github/ShreshthSaxena/ML/blob/master/Youtube_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-api-python-client

     |████████████████████████████████| 61kB 2.4MB/s 
  Found existing installation: google-api-python-client 1.6.7
    Uninstalling google-api-python-client-1.6.7:
      Successfully uninstalled google-api-python-client-1.6.7


In [0]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import pprint 
import matplotlib.pyplot as plt

DEVELOPER_KEY = "API_KEY"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

### **Query Search in Youtube**

In [0]:
def youtube_search(q, max_results=50,order="relevance", token=None, location=None, location_radius=None):

    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

    search_response = youtube.search().list(
    q=q,
    type="video",
    pageToken=token,
    order = order,
    part="id,snippet", # Part signifies the different types of data you want 
    maxResults=max_results,
    location=location,
    locationRadius=location_radius).execute()

    title = []
    channelId = []
    channelTitle = []
    categoryId = []
    videoId = []
    viewCount = []
    likeCount = []
    dislikeCount = []
    commentCount = []
    favoriteCount = []
    category = []
    tags = []
    videos = []
    
    for search_result in search_response.get("items", []):
      if search_result["id"]["kind"] == "youtube#video":
        title.append(search_result['snippet']['title']) 
        videoId.append(search_result['id']['videoId'])

        response = youtube.videos().list(
            part='statistics, snippet',
            id=search_result['id']['videoId']).execute()

        channelId.append(response['items'][0]['snippet']['channelId'])
        channelTitle.append(response['items'][0]['snippet']['channelTitle'])
        categoryId.append(response['items'][0]['snippet']['categoryId'])
        favoriteCount.append(response['items'][0]['statistics']['favoriteCount'])
        viewCount.append(response['items'][0]['statistics']['viewCount'])
        likeCount.append(response['items'][0]['statistics']['likeCount'])
        dislikeCount.append(response['items'][0]['statistics']['dislikeCount'])
      if 'commentCount' in response['items'][0]['statistics'].keys():
        commentCount.append(response['items'][0]['statistics']['commentCount'])
      else:
        commentCount.append([])

      if 'tags' in response['items'][0]['snippet'].keys():
        tags.append(response['items'][0]['snippet']['tags'])
      else:
        tags.append([])

    youtube_dict = {'tags':tags,'channelId': channelId,'channelTitle': channelTitle,'categoryId':categoryId,'title':title,'videoId':videoId,'viewCount':viewCount,'likeCount':likeCount,'dislikeCount':dislikeCount,'commentCount':commentCount,'favoriteCount':favoriteCount}

    return youtube_dict


In [5]:
hw = youtube_search("John Butler")
hw.keys()

dict_keys(['tags', 'channelId', 'channelTitle', 'categoryId', 'title', 'videoId', 'viewCount', 'likeCount', 'dislikeCount', 'commentCount', 'favoriteCount'])

In [6]:
d = pd.DataFrame(data = hw)
d1 = d[['title','viewCount','channelTitle','commentCount','likeCount','dislikeCount','tags','favoriteCount','videoId','channelId','categoryId']]
d1.columns = ['Title','viewCount','channelTitle','commentCount','likeCount','dislikeCount','tags','favoriteCount','videoId','channelId','categoryId']
del d
d1.head()

,Title,viewCount,channelTitle,commentCount,likeCount,dislikeCount,tags,favoriteCount,videoId,channelId,categoryId
0,John Butler ‘Discovering Stillness – Part 1’ I...,1395916,conscioustv,1283,12098,490,"[John Butler, Mystic approaches, Wonders of sp...",0,fZXhPmHPoNo,UCtSQfqA-Sv3DMScpk32vbIg,27
1,The Lanzarote Tapes - #4 The &#39;couldn&#39;t...,13764,Spiritual Unfoldment with John Butler,75,312,10,"[asmr, awakening, bakewell, cesar manrique, ch...",0,UYnj70LWzxA,UCi0TFuqj6eND-mJRf8i2Tnw,22
2,OCEAN - John Butler - 2012 Studio Version,43093647,John Butler Trio,20394,403569,6435,"[Ocean, John Butler, Trio, Live, 2012, revolut...",0,jdYJf_ybyVo,UCQ0gZgLbqHywkNU_mDJKDzg,10
3,What is Truth?,36444,Spiritual Unfoldment with John Butler,227,639,21,"[John Butler, Bakewell, spiritual unfoldment, ...",0,sU9Tzh5jw6Y,UCi0TFuqj6eND-mJRf8i2Tnw,22
4,John Butler asks &quot;What is salvation?&quot;,52059,Spiritual Unfoldment with John Butler,262,652,35,"[John Butler, Bakewell, meditation, salvation,...",0,WZ_4zn7tFVU,UCi0TFuqj6eND-mJRf8i2Tnw,22


In [7]:
d1[d1['channelTitle'] == 'John Butler Trio']

,Title,viewCount,channelTitle,commentCount,likeCount,dislikeCount,tags,favoriteCount,videoId,channelId,categoryId
2,OCEAN - John Butler - 2012 Studio Version,43093647,John Butler Trio,20394,403569,6435,"[Ocean, John Butler, Trio, Live, 2012, revolut...",0,jdYJf_ybyVo,UCQ0gZgLbqHywkNU_mDJKDzg,10
10,John Butler - Used To Get High For A Living,4191920,John Butler Trio,748,13690,353,"[John, Butler, Trio, Used, To, Get, High, For,...",0,yI0_w9aV8G0,UCQ0gZgLbqHywkNU_mDJKDzg,10
18,John Butler Trio - &#39;Wade In The Water&#39;...,592414,John Butler Trio,321,5962,148,"[John Butler, John Butler Trio, Wade In The Wa...",0,85DKN7j5vW8,UCQ0gZgLbqHywkNU_mDJKDzg,1
21,John Butler Trio - &#39;Tell Me Why&#39; (Offi...,466182,John Butler Trio,189,5014,123,"[John Butler, John Butler Trio, John Butler Tr...",0,zisQiShKrJQ,UCQ0gZgLbqHywkNU_mDJKDzg,1
24,John Butler Trio &quot;Young &amp; Wild&quot; ...,3624620,John Butler Trio,392,15476,380,"[John Butler, John Butler Trio, JBT, Flesh & B...",0,_ukGTIYcJ4w,UCQ0gZgLbqHywkNU_mDJKDzg,10
27,John Butler Trio - Just Call (Official Music V...,599129,John Butler Trio,201,4719,144,"[Just Call, HOME, John Butler, John Butler Tri...",0,djEqLRdqScM,UCQ0gZgLbqHywkNU_mDJKDzg,1
29,John Butler Trio &quot;Spring to Come&quot; Ac...,5807106,John Butler Trio,547,33364,595,"[The John Butler Trio (Musical Group), John Bu...",0,uiBxYDyyb14,UCQ0gZgLbqHywkNU_mDJKDzg,10
34,Zebra - Official Video,5334511,John Butler Trio,857,24454,545,"[zebra, john, butler, trio, sunrise, over, sea...",0,PO2b3cggqs0,UCQ0gZgLbqHywkNU_mDJKDzg,10
41,Fool For You - John Butler Trio - Official Video,1493377,John Butler Trio,212,6900,106,"[John Butler Trio, JBT, John, Butler, Fool For...",0,v1_56godVIc,UCQ0gZgLbqHywkNU_mDJKDzg,10
47,John Butler Trio Only One Acoustic In Studio,1199583,John Butler Trio,112,3932,57,"[john butler, Acoustic Guitar (Musical Instrum...",0,s8YQWRlRxHY,UCQ0gZgLbqHywkNU_mDJKDzg,10


### **Getting Comments for a Video**

In [8]:
def get_comment_threads(youtube, video_id, comments):
  threads = []
  results = youtube.commentThreads().list(
     part="snippet",
     videoId=video_id,
     textFormat="plainText",
   ).execute()

  #Get the first set of comments
  for item in results["items"]:
    threads.append(item)
    comment = item["snippet"]["topLevelComment"]
    text = comment["snippet"]["textDisplay"]
    comments.append(text)

  #Keep getting comments from the following pages
  while ("nextPageToken" in results):
    results = youtube.commentThreads().list(
      part="snippet",
      videoId=video_id,
      pageToken=results["nextPageToken"],
      textFormat="plainText",
    ).execute()
    for item in results["items"]:
      threads.append(item)
      comment = item["snippet"]["topLevelComment"]
      text = comment["snippet"]["textDisplay"]
      comments.append(text)

  print ("Total threads: ",len(threads))

  return threads


def get_comments(youtube, parent_id, comments):
  results = youtube.comments().list(
    part="snippet",
    parentId=parent_id,
    textFormat="plainText"
  ).execute()

  for item in results["items"]:
    text = item["snippet"]["textDisplay"]
    comments.append(text)

  return results["items"]

  
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
comments = []
video_comment_threads = get_comment_threads(youtube, 'PO2b3cggqs0', comments)          #Change videoId here

for thread in video_comment_threads:
  get_comments(youtube, thread["id"], comments)
print ('Total comments: ',len(comments))

com = pd.DataFrame(data = comments, columns ={'Comments'})

Total threads:  728
Total comments:  857


## **VADER**

train and build your own sentiment analyser with the NaiveBayesClassifier class. https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/

In [0]:
!pip install twython

In [9]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [10]:
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# sid = SentimentIntensityAnalyzer()
# def get_vader_score(sent):
#     # Polarity score returns dictionary
#     ss = sid.polarity_scores(sent)
#     for k in sorted(ss):
#         print('{0}: {1}, '.format(k, ss[k]), end='')
#         print()

#To iterate the dataset
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def nltk_sentiment(sentence):    
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [11]:
#RESULTS

nltk_results = [nltk_sentiment(row) for row in com['Comments']]
results_df = pd.DataFrame(nltk_results)
text_df = pd.DataFrame(data = com)
nltk_df = text_df.join(results_df)

nltk_df

,Comments,compound,neg,neu,pos
0,Watched this today at film school. Great video,0.6249,0.000,0.631,0.369
1,Watched this today at film school. Great video,0.6249,0.000,0.631,0.369
2,Watched this today at film school. Great video,0.6249,0.000,0.631,0.369
3,This is most bogus song ever by butler ☹️,0.0000,0.000,1.000,0.000
4,I love this song.,0.6369,0.000,0.323,0.677
5,Ahaa i found the song!! I have been looking fo...,-0.2924,0.072,0.879,0.049
6,why is a trio one guy,0.0000,0.000,1.000,0.000
7,I didn't get the damn job I was really looking...,0.8924,0.000,0.620,0.380
8,love your songs,0.6369,0.000,0.323,0.677
9,2019 yallllllll,0.0000,0.000,1.000,0.000


In [12]:
pos = neg = neu = 0

for score in nltk_df['compound']:
  if score > 0.2:
    pos = pos+1
  elif score < -0.2:
    neg = neg+1
  else:
    neu = neu+1
total = nltk_df.shape[0]
print("positive ",pos*100/total, " percent") 
print("negative ",neg*100/total, " percent")
print("neutral ",neu*100/total," percent")

positive  48.65810968494749  percent
negative  10.968494749124854  percent
neutral  40.37339556592765  percent


## **TextBlob Polarities**

In [13]:
!pip install textblob
from textblob import TextBlob
# Get the polarity score using below function
def get_textBlob_score(sent):
    # This polarity score is between -1 to 1
    polarity = TextBlob(sent).sentiment.polarity
    return polarity

In [39]:
tb_results = [get_textBlob_score(row) for row in com['Comments']]
res_df = pd.DataFrame(tb_results)
text_df = pd.DataFrame(data = com)
tb_df = text_df.join(res_df)
tb_df.columns = ['Comments','Score']
tb_df

,Comments,Score
0,Watched this today at film school. Great video,0.800000
1,Watched this today at film school. Great video,0.800000
2,Watched this today at film school. Great video,0.800000
3,This is most bogus song ever by butler ☹️,0.500000
4,I love this song.,0.500000
5,Ahaa i found the song!! I have been looking fo...,0.025000
6,why is a trio one guy,0.000000
7,I didn't get the damn job I was really looking...,0.383333
8,love your songs,0.500000
9,2019 yallllllll,0.000000


In [40]:
print("total comments: ",tb_df.shape[0])
print("positive scores: ",tb_df[tb_df['Score']>0].shape[0])
print("negative scores: ",tb_df[tb_df['Score']<0].shape[0])
print("neutral scores: ",tb_df[tb_df['Score']==0].shape[0])

total comments:  857
positive scores:  399
negative scores:  101
neutral scores:  357


From the large number of neutral scores we can conclude that we need a better classification technique !